In [1]:
print("Hello")

Hello


In [7]:
import requests
import json
from dateutil import parser

In [8]:
import pandas as pd

In [9]:
API_KEY = "dc6b098a0c20ab9673b63f8ad72b81d7-758b1a37d2ef19d1eb3510cc195802b6"
ACCOUNT_ID = "101-001-31068424-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [10]:
session = requests.Session()

In [11]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [12]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [13]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [14]:
response = session.get(url, params = params, data = None, headers = None)

In [15]:
response.status_code

200

In [16]:
data = response.json()

In [17]:
instruments_list = data['instruments']

In [18]:
len(instruments_list)

68

In [19]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [20]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [21]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i}

In [22]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [23]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent = 2))

In [24]:
with open("../exploration/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent = 2))

In [25]:
def fetch_candles(pair_name, count = 10, granularity = "H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params = params, data = None, headers = None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    # mid_c, mid_o, bid_h

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['volume'] = candle['volume']
        new_dict['time'] = parser.parse(candle['time'])
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count = 10, granularity = "H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [26]:
code, data = fetch_candles("EUR_USD", count = 10, granularity = "H4")
candles_df = get_candles_df(data)

In [27]:
create_data_file("EUR_USD", count = 10, granularity = "H4")

EUR_USD H4 9 candles, 2025-02-20 02:00:00+00:00 2025-02-21 10:00:00+00:00


In [28]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   volume  9 non-null      int64                  
 1   time    9 non-null      datetime64[ns, tzutc()]
 2   mid_o   9 non-null      float64                
 3   mid_h   9 non-null      float64                
 4   mid_l   9 non-null      float64                
 5   mid_c   9 non-null      float64                
 6   bid_o   9 non-null      float64                
 7   bid_h   9 non-null      float64                
 8   bid_l   9 non-null      float64                
 9   bid_c   9 non-null      float64                
 10  ask_o   9 non-null      float64                
 11  ask_h   9 non-null      float64                
 12  ask_l   9 non-null      float64                
 13  ask_c   9 non-null      float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), in

In [29]:
code

200

In [30]:
len(data)

10

In [31]:
data

[{'complete': True,
  'volume': 17180,
  'time': '2025-02-20T02:00:00.000000000Z',
  'bid': {'o': '1.04206', 'h': '1.04369', 'l': '1.04195', 'c': '1.04269'},
  'mid': {'o': '1.04214', 'h': '1.04376', 'l': '1.04203', 'c': '1.04276'},
  'ask': {'o': '1.04222', 'h': '1.04385', 'l': '1.04211', 'c': '1.04284'}},
 {'complete': True,
  'volume': 22500,
  'time': '2025-02-20T06:00:00.000000000Z',
  'bid': {'o': '1.04269', 'h': '1.04414', 'l': '1.04227', 'c': '1.04364'},
  'mid': {'o': '1.04278', 'h': '1.04421', 'l': '1.04236', 'c': '1.04372'},
  'ask': {'o': '1.04286', 'h': '1.04428', 'l': '1.04244', 'c': '1.04379'}},
 {'complete': True,
  'volume': 25358,
  'time': '2025-02-20T10:00:00.000000000Z',
  'bid': {'o': '1.04363', 'h': '1.04541', 'l': '1.04300', 'c': '1.04475'},
  'mid': {'o': '1.04370', 'h': '1.04550', 'l': '1.04308', 'c': '1.04482'},
  'ask': {'o': '1.04377', 'h': '1.04558', 'l': '1.04315', 'c': '1.04490'}},
 {'complete': True,
  'volume': 37266,
  'time': '2025-02-20T14:00:00.000

In [32]:
data[0]

{'complete': True,
 'volume': 17180,
 'time': '2025-02-20T02:00:00.000000000Z',
 'bid': {'o': '1.04206', 'h': '1.04369', 'l': '1.04195', 'c': '1.04269'},
 'mid': {'o': '1.04214', 'h': '1.04376', 'l': '1.04203', 'c': '1.04276'},
 'ask': {'o': '1.04222', 'h': '1.04385', 'l': '1.04211', 'c': '1.04284'}}

In [33]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']
# mid_c, mid_o, bid_h

In [34]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [35]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count = 4001, granularity = g)

EUR_USD H1 4000 candles, 2024-07-01 23:00:00+00:00 2025-02-21 15:00:00+00:00
EUR_USD H4 4000 candles, 2022-07-27 21:00:00+00:00 2025-02-21 10:00:00+00:00
EUR_GBP H1 4000 candles, 2024-07-01 23:00:00+00:00 2025-02-21 15:00:00+00:00
EUR_GBP H4 4000 candles, 2022-07-27 21:00:00+00:00 2025-02-21 10:00:00+00:00
EUR_JPY H1 4000 candles, 2024-07-01 23:00:00+00:00 2025-02-21 15:00:00+00:00
EUR_JPY H4 4000 candles, 2022-07-27 21:00:00+00:00 2025-02-21 10:00:00+00:00
EUR_CHF H1 4000 candles, 2024-07-01 23:00:00+00:00 2025-02-21 15:00:00+00:00
EUR_CHF H4 4000 candles, 2022-07-27 21:00:00+00:00 2025-02-21 10:00:00+00:00
EUR_NZD H1 4000 candles, 2024-07-01 22:00:00+00:00 2025-02-21 15:00:00+00:00
EUR_NZD H4 4000 candles, 2022-07-28 05:00:00+00:00 2025-02-21 10:00:00+00:00
EUR_CAD H1 4000 candles, 2024-07-01 23:00:00+00:00 2025-02-21 15:00:00+00:00
EUR_CAD H4 4000 candles, 2022-07-27 21:00:00+00:00 2025-02-21 10:00:00+00:00
EUR_AUD H1 4000 candles, 2024-07-01 23:00:00+00:00 2025-02-21 15:00:00+00:00